In [ ]:
import eikon as ek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ek.set_app_key('89915a3b58874e1599870c6ecc45d6edd6344f8c')

In [ ]:
CLOSEPRICE_df,err = ek.get_data(instruments='0#.STOXX',fields=['TR.CLOSEPRICE','TR.CLOSEPRICE.periodenddate'],parameters={'SDate':'20000101','EDate':'20230101','Frq':'CM','Curn':'EUR','CALCMETHOD':'CLOSE'})

In [ ]:
CLOSEPRICE_df.rename(columns={'Period End Date':'Date'},inplace=True)

In [ ]:
CLOSEPRICE_df

In [ ]:
sns.lineplot(data=CLOSEPRICE_df, x='Date',y='Close Price',hue='Instrument',legend=False)

In [ ]:
pivoted_CLOSEPRICE_df = CLOSEPRICE_df.pivot(index='Date',columns='Instrument',values='Close Price')

In [ ]:
pivoted_CLOSEPRICE_df

In [ ]:
selected_constituents_df = pivoted_CLOSEPRICE_df.dropna(axis=1)

In [ ]:
ranked_df = selected_constituents_df.T

In [ ]:
###### HAY QUE HACER DE FORMA VECTORIZADA #########
for date,prices in zip(selected_constituents_df.index,selected_constituents_df.astype(float).values):
    ranked_df[date] = pd.qcut(prices,10,duplicates='drop',labels=False)
ranked_df

In [ ]:
#### decil 0 tiene los valores mas bajos y el 9 los mas altos ####
def deciles_lists(df):
    diccionario = {}
    for i in range(10):
        diccionario[f'decil_{i}'] = {}
        for date,ranks in df.items():
            diccionario[f'decil_{i}'][date] = ranks.loc[ranks == i].index.tolist()
    return diccionario

In [ ]:
deciles_dict = deciles_lists(ranked_df)

In [ ]:
rentabilidad_acciones_df = selected_constituents_df.pct_change()

In [ ]:
deciles_df = selected_constituents_df.quantile(q=np.arange(0.1,1,0.1),axis=1)

In [ ]:
rentabilidad_acciones_df

In [ ]:
rentabilidades_dict = {}
rentabilidades_dict['equiponderado'] = {}
for decil,fechas in deciles_dict.items():
    rentabilidades_dict[decil] = {}
    for fecha, stocks in fechas.items():
        rentabilidades_dict[decil][fecha] = rentabilidad_acciones_df.loc[fecha,stocks].mean()
        rentabilidades_dict['equiponderado'][fecha] = rentabilidad_acciones_df.loc[fecha].mean()

In [ ]:
plt.style.use('ggplot')

In [ ]:
plt.figure(figsize=(20,10))
for decil in rentabilidades_dict:
    plt.plot(list(rentabilidades_dict['decil_0'].keys())[1:],np.array(list(rentabilidades_dict[decil].values())[1:]).cumsum(),label=decil)
plt.xticks(rotation=-45,fontsize=10,ha='left',rotation_mode='anchor')
plt.legend()
plt.tight_layout()

In [1]:
keys = list(rentabilidades_dict.keys())
rentabilidades_medias = [np.mean(list(rentabilidades_dict[decil].values())[1:]) for decil in rentabilidades_dict]

plt.bar(keys[::-1],rentabilidades_medias[::-1])
plt.xticks(rotation=-45,ha='left',rotation_mode='anchor')

NameError: name 'rentabilidades_dict' is not defined